# 对极几何

<p>
<img width=500 src="./images/对极约束.png"/>
</p>

## 求解本质矩阵

在不考虑畸变的情况下，经典成像模型为：

$$
{}^cZ{}^ip=K{}^cT_w{}^wP
$$

考虑一个相机在两个不同位置对一个点进行成像，有：

$$
\begin{array}{l}
{}^cZ_1{}^ip_1=K{}^{c_1}T_w{}^wP \\
{}^cZ_2{}^ip_2=K{}^{c_2}T_{c_1}{}^{c_1}T_w{}^wP
\end{array}
$$

将第一个相机的位姿定义为世界坐标系原点，则${}^{c_1}T_w$为单位矩阵，则有：

$$
\begin{array}{l}
{}^cZ_1{}^ip_1=K{}^wP \\
{}^cZ_2{}^ip_2=K{}^{c_2}T_{c_1}{}^wP
\end{array} \Longrightarrow 
\begin{array}{l}
x_1 = K^{-1}{}^cZ_1{}^ip_1={}^wP \\
x_2 = K^{-1}{}^cZ_2{}^ip_2={}^{c_2}T_{c_1}{}^wP
\end{array}
$$

最终得到约束：

$$
x_2^{\land}x_2=x_2^{\land}{}^{c_2}T_{c_1}{}x_1=0
$$

求解此线性方程组即可得到第二个相机的本质矩阵。注意到常数对此约束不带来任何影响。

In [1]:
import numpy as np
from gray_video import gray_video
from core import orb_detector
from skimage.measure import ransac
from skimage.transform import EssentialMatrixTransform

In [2]:
# 相机内参
K = np.array([[1.20981082e+03, -6.38606499e-01,  6.27897309e+02],
              [0.00000000e+00,  1.18458039e+03,  3.30943028e+02],
              [0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
print("相机内参为：\n{}".format(K))


相机内参为：
[[ 1.20981082e+03 -6.38606499e-01  6.27897309e+02]
 [ 0.00000000e+00  1.18458039e+03  3.30943028e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [3]:
# 理论复现方法
def resolve_essential(matched_points: np.ndarray, K: np.ndarray) -> np.ndarray:
    K_inv = np.linalg.inv(K)
    homogeneous_coordinates = np.ones((len(matched_points), 2, 3))
    homogeneous_coordinates[:, :, :2] = matched_points
    # 使用相机内参对角点坐标归一化
    norm_curr_kps = (K_inv @ homogeneous_coordinates[:, 0].T)[:2].T
    norm_last_kps = (K_inv @ homogeneous_coordinates[:, 1].T)[:2].T
    # 求解本质矩阵和内点数据
    model, inliers = ransac((norm_curr_kps, norm_last_kps),
                            EssentialMatrixTransform,
                            min_samples=8,              # 最少需要 8 个点
                            residual_threshold=0.0001,
                            max_trials=50000)

    E = model.params
    # 返回本质矩阵
    return E


In [4]:
import cv2


def cv2_resolve_essential(img1, img2, K):
    # opencv封装好的方法
    orb = cv2.ORB_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts2 = np.float32(pts2)
    pts1 = np.float32(pts1)

    E, mask = cv2.findEssentialMat(pts1, pts2, K)
    return E


In [5]:
matched_points = orb_detector.cross_detect(gray_video[0], gray_video[100])
E_0 = resolve_essential(matched_points, K)
E_1 = cv2_resolve_essential(gray_video[0], gray_video[100], K)


In [6]:
E_0 /= E_0[-1, -1]
E_1 /= E_1[-1, -1]
print(E_0)
print(E_1)

[[   1.68373832  233.72800473   21.19205453]
 [-233.60007895    1.79298981  -30.73127592]
 [ -12.45399942   29.5034494     1.        ]]
[[   2.24475872  269.28355373   29.96756523]
 [-269.93929493    3.53149948  -34.68363781]
 [ -20.61488766   33.37890993    1.        ]]


可以看到使用理论推导计算出的本质矩阵与**opencv**封装之后的函数计算出的本质矩阵有些许差别，这些差别来源于：
1. 特征点的选取不同
2. 重复采样一致性原理本身具有的不确定性
3. 优化算法迭代次数的不同
但是两者具体的数值分布具有一致性。

## 从本质矩阵恢复相机位姿

从本质矩阵恢复相机位姿一般用本质矩阵的SVD分解得到。

$$
E=U\Sigma{}V^T
$$

设:

$$
W=\begin{bmatrix}
    0 & -1 & 0 \\
    1 & 0 & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$$

有：

$$
\begin{array}{l}
     R_1 =& UWV^T\\
     R_2 =& UW^TV^T
\end{array}
$$

$t$为特征值分解后最小特征值对应的$U$矩阵中向量，其中$U$、$V$需正定

In [7]:
def decompose_E(E: np.ndarray):
    W = np.array(
        [[0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]]
    )
    U, S, Vt = np.linalg.svd(E)
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    if np.linalg.det(R1) < 0:
        R1 = -R1
    if np.linalg.det(R2) < 0:
        R2 = -R2
    t = U.T[-1]
    return R1,R2,t

In [8]:
R1, R2, t = decompose_E(E_0)
print("候选旋转矩阵1：\n{}".format(R1))
print("候选旋转矩阵2：\n{}".format(R2))
print("候选平移向量：\n{}".format(t))


候选旋转矩阵1：
[[ 0.99996993 -0.00544686  0.00551988]
 [ 0.00524497  0.99933977  0.03595145]
 [-0.00571206 -0.03592142  0.99933829]]
候选旋转矩阵2：
[[-0.96741476  0.02753301 -0.25169548]
 [ 0.00878486 -0.98982163 -0.14204214]
 [-0.25304448 -0.13962477  0.95732618]]
候选平移向量：
[-0.12464957 -0.0537184   0.99074559]


## 选择正确的旋转矩阵与平移向量

用计算后的位姿转换矩阵将匹配点重投影到世界坐标中，如果所有的坐标深度都为正，那这个位姿转换矩阵所对应的旋转矩阵与平移向量则最终确定为正确的旋转矩阵与平移向量。

### 求解匹配点深度

$$
{}^ip^\land{}{}^ip={}^ip^{\land}KT{}^wP=0 \\
{}^ip^\land=\begin{bmatrix}
    0 & -1 & v \\
    1 & 0 & -u \\
    -v & u & 0 \\
\end{bmatrix}
$$

设$KT=G$，存在约束：

$$
\begin{bmatrix}
    0 & -1 & v \\
    1 & 0 & -u \\
    -v & u & 0 \\
\end{bmatrix}G{}^wP=0
$$

${}^ip^\land$的秩为$2$，每个点可以为求解${}^wP$带来两个约束，使用匹配的两个点即可求得${}^wP$

In [9]:
normalized_K = np.eye(4)
normalized_K[:3,:3] = K
normalized_K_inv = np.linalg.inv(normalized_K)

#### 使用三角测量计算深度

In [10]:
def triangulate(matched_points, G1, G2):
    p3ds = np.zeros((len(matched_points), 3))
    for i in range(len(matched_points)):
        point1 = matched_points[i][0]
        point2 = matched_points[i][1]
        A = np.zeros((4, 4))
        A[0] = point1[0] * G1[2] - G1[0]
        A[1] = point1[1] * G1[2] - G1[1]
        A[2] = point2[0] * G2[2] - G2[0]
        A[3] = point2[1] * G2[2] - G2[1]
        U, S, Vt = np.linalg.svd(A)
        p4d = Vt[-1]
        p3ds[i] = (p4d / p4d[-1])[:3].T
    return p3ds

#### 验证R、T

In [11]:
def generate_G(R,t,K):
    T = np.zeros((3, 4))
    T[:,:3]= R
    T[:,3]= t
    G = K@T
    return G

In [12]:
def check_Rt(R, t, K, matched_points):
    # 第一个相机的位姿
    R1 = np.eye(3)
    t1 = np.zeros((3))
    G1 = generate_G(R1, t1, K)
    # 第二个相机的位姿
    G2 = generate_G(R, t, K)
    # 三角测量算出匹配点在第一个相机下的世界坐标
    p3ds1 = triangulate(matched_points, G1, G2)
    # 计算匹配点在第二个相机下的世界坐标
    p3ds2 = (R @ p3ds1.T).T + t

    # 第一个相机下的匹配点世界坐标深度大于个数
    first_good_num = (p3ds1[:, -1] < 0).sum()
    # 第二个相机下的匹配点世界坐标深度是否大部分都大于0
    second_good_num = (p3ds2[:, -1] < 0).sum()

    return (first_good_num, second_good_num)


In [13]:
print("匹配点总对数为：")
print(len(matched_points))

匹配点总对数为：
148


In [14]:
result = check_Rt(R1,t,K,matched_points)
print("对于R1,t计算出深度为负的个数为：")
print(result)

对于R1,t计算出深度为负的个数为：
(148, 147)


In [15]:
result = check_Rt(R1,-t,K,matched_points)
print("对于R1,-t计算出深度为负的个数为：")
print(result)

对于R1,-t计算出深度为负的个数为：
(0, 1)


In [16]:
result = check_Rt(R2,t,K,matched_points)
print("对于R2,t计算出深度为负的个数为：")
print(result)

对于R2,t计算出深度为负的个数为：
(148, 1)


In [17]:
result = check_Rt(R2,-t,K,matched_points)
print("对于R2,-t计算出深度为负的个数为：")
print(result)

对于R2,-t计算出深度为负的个数为：
(0, 147)


对于$R_2$、$-t$来说，计算得出世界坐标下点值为负数的个数最少，因此选择这一对参数作为相机外参。

In [18]:
print("R2为：")
print(R2)
print("-t为：")
print(-t)

R2为：
[[-0.96741476  0.02753301 -0.25169548]
 [ 0.00878486 -0.98982163 -0.14204214]
 [-0.25304448 -0.13962477  0.95732618]]
-t为：
[ 0.12464957  0.0537184  -0.99074559]


In [19]:
a,b,c,d = np.random.randn((4))

In [20]:
np.linalg.matrix_rank(np.array(
    [[b,-a,0,0],
    [0,c,-b,0],
    [0,0,d,-c],
    [-d,0,0,a]]
))

3

In [21]:
def resolve_T(K, matched_points):
    normalized_K = np.eye(4)
    normalized_K[:3,:3] = K
    normalized_K_inv = np.linalg.inv(normalized_K)

    A = np.zeros((len(matched_points)*6, 13))
    for i in range(len(matched_points)):
        x1 = np.ones(4)
        x2 = np.ones(4)
        x1[:2] = matched_points[i][0]
        x2[:2] = matched_points[i][1]
        x1 = normalized_K_inv@x1
        x2 = normalized_K_inv@x2
        a, b, c, d = x1
        e, f, g, h = x2
        A[i*6] = np.array([a*f, b*f, c*f, -a*e, -b*e, -c*e, 0, 0, 0, b*f, -d*e, 0, 0])
        A[i*6+1] = np.array([0, 0, 0, a*g, b*g, c*g, -a*f, -b*f, -c*f, 0, d*g, -d*f, 0])
        A[i*6+2] = np.array([0, 0, 0, 0, 0, 0, a*h, b*h, c*h, 0, 0, d*h, -d*g])
        A[i*6+3] = np.array([-a*h, -b*h, -c*h, 0, 0, 0, 0, 0, 0, -d*h, 0, 0, d*e])
        A[i*6+4] = np.array([a*g, b*g, c*g, 0, 0, 0, -a*e, -b*e, -c*e, d*g, 0, -d*e, 0])
        A[i*6+5] = np.array([0, 0, 0, a*h, b*h, c*h, 0, 0, 0, 0, d*h, 0, -d*f])
    U, S, Vt = np.linalg.svd(A)
    T = np.eye(4)
    T[:3] = Vt[-1][:12].reshape((3,4)) / Vt[-1][-1]
    return T


In [22]:
normalized_K = np.eye(4)
normalized_K[:3,:3] = K
normalized_K_inv = np.linalg.inv(normalized_K)
A = np.zeros((len(matched_points)*6, 13))
for i in range(len(matched_points)):
    x1 = np.ones(4)
    x2 = np.ones(4)
    x1[:2] = matched_points[i][0]
    x2[:2] = matched_points[i][1]
    x1 = normalized_K_inv@x1
    x2 = normalized_K_inv@x2
    a, b, c, d = x1
    e, f, g, h = x2
    A[i*6] = np.array([a*f, b*f, c*f, -a*e, -b*e, -c*e, 0, 0, 0, b*f, -d*e, 0, 0])
    A[i*6+1] = np.array([0, 0, 0, a*g, b*g, c*g, -a*f, -b*f, -c*f, 0, d*g, -d*f, 0])
    A[i*6+2] = np.array([0, 0, 0, 0, 0, 0, a*h, b*h, c*h, 0, 0, d*h, -d*g])
    A[i*6+3] = np.array([-a*h, -b*h, -c*h, 0, 0, 0, 0, 0, 0, -d*h, 0, 0, d*e])
    A[i*6+4] = np.array([a*g, b*g, c*g, 0, 0, 0, -a*e, -b*e, -c*e, d*g, 0, -d*e, 0])
    A[i*6+5] = np.array([0, 0, 0, a*h, b*h, c*h, 0, 0, 0, 0, d*h, 0, -d*f])

U, S, Vt = np.linalg.svd(A)
T = np.eye(4)
R = Vt[-3]
R /= R[-1]
T[:3,:3] = R[:9].reshape((3,3))
T[:3,-1] = R[9:12]
T[-1,-1] = R[-1]

In [23]:
A.shape

(888, 13)

In [24]:
np.linalg.matrix_rank(A)

11

In [25]:
x = np.linalg.lstsq(A, np.zeros(A.shape[0]),rcond=None)

In [26]:
x

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([], dtype=float64),
 11,
 array([2.46266131e+01, 2.43765448e+01, 2.13651774e+01, 4.26076065e+00,
        3.20684352e+00, 2.28810900e+00, 1.26286032e+00, 1.01243080e+00,
        7.28347139e-01, 7.22895712e-01, 4.18795852e-01, 4.00262054e-15,
        3.04781593e-15]))